<a href="https://colab.research.google.com/github/alkang53/CNN-BASED-TEXT-CLASSIFICATION/blob/main/fromcsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from scipy.io import arff
import pandas as pd
import csv

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling2D, Dropout, Input, Embedding
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#TTC4900 - kelimelerin orj halini içeren csv
csv_file = '/content/drive/My Drive/7allV03.csv'
data = pd.read_csv(csv_file)

corpus = data['text']
categoryNum = 7

In [ ]:
#TTC4900 - kelimelerin köklerini içeren csv
csv_file = '/content/drive/My Drive/7allV03_lem.csv'
data = pd.read_csv(csv_file)

corpus = data['text']
categoryNum = 7

In [ ]:
#müşteri yorumları - kelimelerin orj halini içeren csv
csv_file = '/content/drive/My Drive/e-ticaret_urun_yorumlari_as.csv'
data = pd.read_csv(csv_file, sep=';')

corpus = data['text']
categoryNum = 2

In [ ]:
#müşteri yorumları - kelimelerin köklerini içeren csv
csv_file = '/content/drive/My Drive/e-ticaret_urun_yorumlari_lem.csv'
data = pd.read_csv(csv_file, sep=';')

corpus = data['text']
categoryNum = 2

In [ ]:
#+ Stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

for indexstop, row in enumerate(corpus):
  tokens = word_tokenize(str(row))
  filtered_text = [t for t in tokens if not t.lower() in stopwords.words("turkish")]
  corpus[indexstop] = " ".join(filtered_text)
  #corpus.replace(str(row), " ".join(filtered_text))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
possible_labels = data.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict
data['label'] = data.category.replace(label_dict)



In [ ]:
#TFIDF
vectorizer = TfidfVectorizer()
xBig = vectorizer.fit_transform(corpus)
y = data['label'].values



In [ ]:
#selectKbest
#Kütüphaneleri yükleme
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#Ki kare testine göre en iyi sonuç veren 2 özelliği seç
features = SelectKBest(chi2, k = 2000)
#features = SelectKBest(k = 2000)
x = features.fit_transform(xBig, y)
#Azaltılmış Özellikler
print('Orjinal Özellik Sayısı:', xBig.shape[1])
print('Seçilmiş Özellik Sayısı:', x.shape[1])

Orjinal Özellik Sayısı: 34710
Seçilmiş Özellik Sayısı: 2000


In [ ]:
#################naive bayes#############################
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x, y):
  #veri seti: 5692 öznitelikten oluşan bir arff dosyası
  modelNB = MultinomialNB()

  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = modelNB.fit(x[train],y[train])

  start = time.process_time();
  y_pred = modelNB.predict(x[test]);
  end = time.process_time();
  ac=accuracy_score(y[test],y_pred);
  acc_per_fold.append(ac);

  cm = confusion_matrix(y[test], y_pred);

  
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

--
Training for fold 1 ...
0.9000441374161089
0.8996436686519108
0.8990925514958412
[[65  3  0  1  2  0  1]
 [ 7 48  1  2  0  2  1]
 [ 3  2 66  0  2  1  2]
 [ 0  2  2 61  2  0  0]
 [ 0  0  1  0 69  1  0]
 [ 0  0  0  0  0 73  0]
 [ 0  4  1  4  1  0 60]]
--
Training for fold 2 ...
0.9199091219157867
0.9190238066815752
0.9180952306038334
[[62  2  3  0  0  0  1]
 [ 5 51  3  3  0  0  2]
 [ 1  0 63  0  0  1  2]
 [ 1  0  0 61  1  0  1]
 [ 1  0  0  0 81  0  0]
 [ 2  1  1  1  0 83  1]
 [ 0  0  2  0  3  0 51]]
--
Training for fold 3 ...
0.8971511124152503
0.8921809776784476
0.8928710838465239
[[77  3  2  0  0  0  0]
 [ 6 52  6  2  1  0  2]
 [ 1  1 68  2  0  2  1]
 [ 6  0  1 52  2  0  1]
 [ 0  0  1  0 60  0  0]
 [ 0  1  1  0  1 72  0]
 [ 1  2  2  2  2  0 57]]
--
Training for fold 4 ...
0.9005852753035899
0.9048735259676352
0.9010884285363744
[[58  3  1  0  1  0  0]
 [ 8 65  5  0  0  0  2]
 [ 4  1 45  0  3  0  1]
 [ 2  0  0 76  0  0  1]
 [ 0  1  3  0 55  0  0]
 [ 0  0  0  0  0 76  0]
 [ 1  1  4  0

In [ ]:
#################random forest#############################

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x, y):
  #veri seti: 5692 öznitelikten oluşan bir arff dosyası
  modelRF = RandomForestClassifier(n_estimators=100)

  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = modelRF.fit(x[train],y[train])

  start = time.process_time();
  y_pred = modelRF.predict(x[test]);
  end = time.process_time();
  ac=accuracy_score(y[test],y_pred);
  acc_per_fold.append(ac);

  cm = confusion_matrix(y[test], y_pred);

  
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

--
Training for fold 1 ...
0.8706370774142945
0.8720029284030996
0.8698461320612629
[[56  6  0  2  3  0  0]
 [ 2 56  2  1  1  0  2]
 [ 4  3 65  4  3  1  5]
 [ 2  0  0 66  1  0  2]
 [ 1  1  0  1 72  0  0]
 [ 0  2  1  1  0 65  1]
 [ 1  2  5  3  0  0 47]]
--
Training for fold 2 ...
0.8964593105864986
0.8964288503569667
0.8962147963395326
[[50  4  2  1  0  1  1]
 [ 4 59  5  0  0  0  0]
 [ 1  5 52  1  2  1  3]
 [ 0  0  1 66  0  0  2]
 [ 0  1  1  0 74  0  3]
 [ 0  0  0  0  0 75  0]
 [ 2  0  2  5  1  0 65]]
--
Training for fold 3 ...
0.8970743607926069
0.8974132297992401
0.8969808073074892
[[53  3  0  0  0  1  1]
 [ 3 61  1  4  0  0  1]
 [ 0  5 54  0  1  0  7]
 [ 1  2  1 67  0  1  1]
 [ 0  0  3  0 68  0  1]
 [ 1  0  0  0  0 66  0]
 [ 0  3  4  1  5  0 70]]
--
Training for fold 4 ...
0.8741523001422836
0.8713388539281396
0.8716350726330993
[[78  5  3  1  2  0  1]
 [ 6 52  2  1  2  0  3]
 [ 4  1 52  0  4  0  2]
 [ 0  1  1 59  1  1  1]
 [ 0  2  0  1 71  0  0]
 [ 1  0  0  1  1 60  0]
 [ 1  4  4  2

In [ ]:
#################  DVM  #############################

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x, y):
  #veri seti: 5692 öznitelikten oluşan bir arff dosyası
  model = svm.SVC(kernel='linear') # Linear Kernel

  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = model.fit(x[train],y[train])

  start = time.process_time();
  y_pred = model.predict(x[test]);
  end = time.process_time();
  ac=accuracy_score(y[test],y_pred);
  acc_per_fold.append(ac);

  cm = confusion_matrix(y[test], y_pred);

  
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

--
Training for fold 1 ...
0.9016405774005591
0.9020264786149648
0.9013317748171643
[[58  2  4  1  0  1  1]
 [ 1 54  3  1  0  0  4]
 [ 2  1 64  2  1  1  6]
 [ 1  1  0 51  1  0  0]
 [ 1  1  2  1 71  0  1]
 [ 1  0  0  0  0 76  0]
 [ 0  1  3  2  1  0 68]]
--
Training for fold 2 ...
0.9091689150326172
0.909401847399135
0.9087313117082214
[[71  2  2  1  0  0  3]
 [ 5 54  3  2  0  1  0]
 [ 3  4 56  0  0  0  1]
 [ 2  0  0 58  0  0  0]
 [ 0  0  2  0 67  0  1]
 [ 0  0  1  2  0 77  0]
 [ 0  1  3  2  1  2 63]]
--
Training for fold 3 ...
0.9159249021000173
0.9137595065163107
0.9140402299757059
[[56  5  2  1  3  0  0]
 [ 3 65  5  2  0  0  0]
 [ 2  1 60  0  1  0  2]
 [ 1  2  1 70  0  0  1]
 [ 0  0  0  0 72  0  0]
 [ 0  1  0  1  1 65  0]
 [ 0  0  4  3  0  0 60]]
--
Training for fold 4 ...
0.9056908866461837
0.9063444446041489
0.9054320049332366
[[64  5  4  2  2  0  0]
 [ 3 81  4  1  2  0  1]
 [ 2  4 49  1  0  0  2]
 [ 0  0  0 64  0  0  1]
 [ 1  0  1  1 62  0  1]
 [ 2  0  0  1  0 60  1]
 [ 0  2  2  1 

In [ ]:
#################  knn  #############################

from sklearn.neighbors import KNeighborsClassifier

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x, y):
  #veri seti: 5692 öznitelikten oluşan bir arff dosyası
  modelKN = KNeighborsClassifier(n_neighbors = 5) # Linear Kernel

  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = modelKN.fit(x[train],y[train])

  start = time.process_time();
  y_pred = modelKN.predict(x[test]);
  end = time.process_time();
  ac=accuracy_score(y[test],y_pred);
  acc_per_fold.append(ac);

  cm = confusion_matrix(y[test], y_pred);

  
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

--
Training for fold 1 ...
0.7493905206958376
0.6784460442313066
0.679866858283036
[[39 16 14 14  0  0  2]
 [ 3 43 14  7  2  1  1]
 [ 0  1 49  8  0  0  2]
 [ 0  1  5 59  1  2  1]
 [ 0  0  9 10 48  0  0]
 [ 1  0 16  6  0 52  0]
 [ 0  1 13  9  2  0 38]]
--
Training for fold 2 ...
0.7822744081459454
0.6759076170064259
0.6854837065590118
[[31  6 15 14  1  0  1]
 [ 1 41 14  9  1  0  0]
 [ 0  4 58  5  0  0  3]
 [ 1  0  5 59  0  1  0]
 [ 0  0 13 18 40  0  2]
 [ 0  1 18  7  0 45  0]
 [ 0  0 16  2  1  0 57]]
--
Training for fold 3 ...
0.796556276257273
0.6867925402094471
0.6983564843481292
[[23  5  6 22  1  0  1]
 [ 2 52  6 21  1  0  3]
 [ 0  1 45 10  1  0  2]
 [ 0  0  1 74  0  0  0]
 [ 0  1  2 12 49  0  0]
 [ 0  1  5 15  0 45  0]
 [ 0  3 10 18  2  0 50]]
--
Training for fold 4 ...
0.7601038407311504
0.6869342185275195
0.6968872850184519
[[30 10 11 20  0  0  0]
 [ 1 44 14  3  2  0  3]
 [ 0  2 64 10  1  4  6]
 [ 0  1  8 61  0  0  0]
 [ 0  1  8  8 53  2  2]
 [ 0  1  7  5  1 46  0]
 [ 1  0 14  7  

In [ ]:
x2 = x.toarray()
#x2 = x2.reshape(x2.shape[0], x2.shape[1], 1)
max_length=x2.shape[1]
vocab_size = x2.shape[1]

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x2, y):
  #veri seti: vocab_size öznitelikten oluşan bir arff dosyası
  modelCNN = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(128, 1, activation="relu", input_shape=(vocab_size,1)), #dropout 0.2 91,5 droopout 0.05 88 dropout 0.3 89
      tf.keras.layers.Flatten(input_shape=(vocab_size,1)),
      tf.keras.layers.Dense(128,activation='relu'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(7)
      ])
  #inputs = Input(shape=(max_length, ))
  #embeddings_layer = Embedding(input_dim=vocab_size, output_dim=128,  input_length=max_length)
  #conv = Conv1D(32, 7, padding="same") ## Channels last
  #dense = Dense(7)
  #embedModel = embeddings_layer(inputs)
  #embedModel = conv(embedModel)
  #embedModel = tf.reduce_max(embedModel, axis=1)
  #output = dense(embedModel)
  #modelCNN = Model(inputs=inputs, outputs=output)


  modelCNN.summary()
  #define loss function variable
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  
  # compile the model
  modelCNN.compile(optimizer='adam',
             loss=loss_fn,
             metrics=['accuracy'])


  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = modelCNN.fit(x2[train],y[train],epochs=10)
  scores = modelCNN.evaluate(x2[test],y[test],verbose=2, batch_size=10)
  print(f'Score for fold {fold_no}: {modelCNN.metrics_names[0]} of {scores[0]}; {modelCNN.metrics_names[1]} of {scores[1]}')
  acc_per_fold.append(scores[1])
  loss_per_fold.append(scores[0])
  
  start = time.process_time();
  y_pred1 = modelCNN.predict(x2[test])
  end = time.process_time();
  print("predictTime: ", end - start)
  y_pred = np.argmax(y_pred1, axis=1)
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  #confusion_matrix(y_true, y_pred)
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 5000, 128)         256       
                                                                 
 flatten (Flatten)           (None, 640000)            0         
                                                                 
 dense (Dense)               (None, 128)               81920128  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 903       
                                                                 
Total params: 81,921,287
Trainable params: 81,921,287
Non-trainable params: 0
_________________________________________________________________
--
Training for fold 1 ...
Epoch 1/10
344/34

In [ ]:
#check am i running at GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'